# 策略概述

30日内存在放量（当天前10），经过一段时间整理后，股价突破放量时的收盘价。

华森制药2021年12月28日

In [152]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsi30, rsiday
from alpha.utils import math_round, buy_limit_price
from numpy.lib.stride_tricks import sliding_window_view

await init_notebook()
init_jq()

In [153]:
def jq_scan(secs, trigger, nbars, tm=None, frequency='daily'):
    fields = ["open", "close", "high", "low", "volume", "high_limit"]
    
    rec_limit_per_req = 3000
    sec_per_req = rec_limit_per_req // nbars
    
    batches = len(secs) // sec_per_req + 1
    
    tm = tm or arrow.now().date()
    results = []
    for i in range(batches):
        codes = secs[i*sec_per_req:(i+1)*sec_per_req]
        if len(codes) == 0:
            break
        bars = jq.get_price(codes, end_date=tm,frequency = frequency, count=nbars, fields=fields, panel=False, skip_paused=True)

        for code in codes:
            bar = bars[bars.code == code]
                
            name = Stock(code).display_name
            trigger(code, name, bar, results)
            
    return results

In [154]:
def tp(code, name, bars, results):
    vol = bars["volume"].values
    close = bars["close"].values
    frame = bars["time"]
    open_ = bars["open"].values
        
    n = len(bars)
    # 区间最大涨幅
    adv = close[-1] / np.min(close) - 1
    if adv < 0 or n < 30 or adv > 1:
        return
    
    # 确定放量时间
    i = np.argmax(vol)
    if close[i] < open_[i]: #放量大阴，不吉
        return
    
    vol_max = np.max(vol)
    if i > 0:
        vol_min = np.min(vol[:i])
        lb = vol_max / vol_min
    else:
        vol_min = np.min(vol[1:])
        lb = vol_max /vol_min
        
    # 确定是否有整理及上穿
    hh = close[i]
    
    i_under = np.argwhere(close[i+1:] < hh).flatten()
    if len(i_under) == 0:
        return
    
    # 调整坐标
    i_under += i + 1
    
    # 如果放量之后一直在整理，未突破，则整理天数应该比(n-i-1)少一天
    if i_under[-1] == 29 or len(i_under) != n - i - 2:
        return

    results.append((code, name, int(lb), adv, len(i_under)))

In [155]:
secs = Stock.choose(['stock'])
results = jq_scan(secs, tp, nbars = 30)
df = pd.DataFrame(results, columns=["code", "name", "lb", "adv", "collect_days"])

/var/folders/b5/73vzvtdn4pn_8wt6rpd2tb_w0000gn/T/ipykernel_90315/2313998961.py:24: RuntimeWarning: divide by zero encountered in double_scalars
  lb = vol_max /vol_min


In [151]:
df[(df.lb>10)].nsmallest(30, "adv")

,code,name,lb,adv,collect_days
47,603718.XSHG,海利生物,16,0.183507,3
7,002034.XSHE,旺能环境,12,0.241771,1
32,600202.XSHG,哈空调,12,0.357143,12
8,002059.XSHE,云南旅游,87,0.644068,1
